In [1]:
import tensorflow as tf
from pathlib import Path

mnist=tf.keras.datasets.mnist.load_data()
(x_train,y_train),(x_test,y_test)=mnist
x_train,x_val=x_train[:-5000],x_train[-5000:]
y_train,y_val=y_train[:-5000],y_train[-5000:]


In [4]:
import requests
import json
import time
import numpy as np

start=time.time()
url="http://192.168.18.29:8501/v1/models/mnist:predict"
data={
    "signature_name":"serving_default",
    "instances":[x_test[0].tolist()]
}
response=requests.post(url,data=json.dumps(data))
probas=response.json()["predictions"][0]
l=[round(p,3) for p in probas]
l=np.array(l)
pred_digit=l.argmax()
prob=l.max()

end=time.time()
print("Time taken(REST API):",np.round(end-start,3)*1e3,"MilliSeconds")
print("Predicted_digit:",pred_digit,"\nProbability:",np.round(prob,3)*100)

Time taken(REST API): 11.0 MilliSeconds
Predicted_digit: 7 
Probability: 99.5


In [5]:
import grpc
import time
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

start=time.time()

VM_IP="192.168.18.29"

channel=grpc.insecure_channel(f"{VM_IP}:8500")
stub=prediction_service_pb2_grpc.PredictionServiceStub(channel)

request=predict_pb2.PredictRequest()
request.model_spec.name="mnist"
request.model_spec.signature_name="serving_default"
x=x_test[0].reshape(1,28,28).astype(np.float32)
request.inputs["keras_tensor_18"].CopyFrom(
    tf.make_tensor_proto(x, dtype=tf.float32)
)
response=stub.Predict(request,timeout=10.0)
probs=response.outputs["output_0"].float_val
probs=np.array(probs)

end=time.time()
pred_digit=probs.argmax()
confidence=probs.max()

print("Time taken(gRPC):",(np.round(end-start,3))*1e3,"MilliSeconds")
print("Predicted Digit:",pred_digit,"\nProbability:",np.round(confidence,3)*100)

Time taken(gRPC): 6.0 MilliSeconds
Predicted Digit: 7 
Probability: 99.5
